#### Overview of Business
There is a small restaurant that sells some fantastic Indian food items. They need our assistance to help the restaurant stay afloat. The restaurant has captured some data from its few months of operation but has yet to learn how to use its data to help them run the business.


Now we want to use the data to answer a few questions about his customers, especially their visiting patterns, the money they’ve spent, and which menu items are the customer’s favorite.

#### DataFrame Creation | PySpark

The restaurant has sales data which contains customers’ information. We have menu data with food items related information and members data with subscriptions related information if any customer has purchased it.

##### We are going to create three data frames. Which contains the following information:-

##### 1. Sales dataframe:- This dataframe contains information related to sales. It has three columns customer_id, order_date, and product_id.

customer_id:- This column contains the id of the customer. Ex:- A, B, C etc.

order_date:- This column includes the item’s ordering dates information- ‘2021-01-11’ etc.

product_id:- This holds the id of the product or food items. Ex:- 1, 2, 3 etc.

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('restaurant_analysis').getOrCreate()

In [3]:
import pyspark.sql.functions as F

In [4]:
# create sales data
sales_data = [
          ('A', '2021-01-01', '1'),
          ('A', '2021-01-01', '2'),
          ('A', '2021-01-07', '2'),
          ('A', '2021-01-10', '3'),
          ('A', '2021-01-11', '3'),
          ('A', '2021-01-11', '3'),
          ('B', '2021-01-01', '2'),
          ('B', '2021-01-02', '2'),
          ('B', '2021-01-04', '1'),
          ('B', '2021-01-11', '1'),
          ('B', '2021-01-16', '3'),
          ('B', '2021-02-01', '3'),
          ('C', '2021-01-01', '3'),
          ('C', '2021-01-01', '1'),
          ('C', '2021-01-07', '3')]
# cols
sales_cols= ["customer_id", "order_date", "product_id"]
sales_df = spark.createDataFrame(data = sales_data, schema = sales_cols)
# view the data
sales_df.show()

+-----------+----------+----------+
|customer_id|order_date|product_id|
+-----------+----------+----------+
|          A|2021-01-01|         1|
|          A|2021-01-01|         2|
|          A|2021-01-07|         2|
|          A|2021-01-10|         3|
|          A|2021-01-11|         3|
|          A|2021-01-11|         3|
|          B|2021-01-01|         2|
|          B|2021-01-02|         2|
|          B|2021-01-04|         1|
|          B|2021-01-11|         1|
|          B|2021-01-16|         3|
|          B|2021-02-01|         3|
|          C|2021-01-01|         3|
|          C|2021-01-01|         1|
|          C|2021-01-07|         3|
+-----------+----------+----------+



In [9]:
sales_df.show()
menu_df.show()

+-----------+----------+----------+
|customer_id|order_date|product_id|
+-----------+----------+----------+
|          A|2021-01-01|         1|
|          A|2021-01-01|         2|
|          A|2021-01-07|         2|
|          A|2021-01-10|         3|
|          A|2021-01-11|         3|
|          A|2021-01-11|         3|
|          B|2021-01-01|         2|
|          B|2021-01-02|         2|
|          B|2021-01-04|         1|
|          B|2021-01-11|         1|
|          B|2021-01-16|         3|
|          B|2021-02-01|         3|
|          C|2021-01-01|         3|
|          C|2021-01-01|         1|
|          C|2021-01-07|         3|
+-----------+----------+----------+

+----------+-------------+-----+
|product_id| product_name|price|
+----------+-------------+-----+
|         1| palak_paneer|  100|
|         2|chicken_tikka|  150|
|         3|   jeera_rice|  120|
|         4|        kheer|  110|
|         5|     vada_pav|   80|
|         6| paneer_tikka|  180|
+----------+------

##### Now we will create the menu dataframe, which has food items related information.

##### 2. Menu dataframe:- This dataframe contains food items related information. It has three columns 'product_id', ‘product_name’, and ‘price.’

product_id:- This column has the id of food items. Ex:- 1, 2, 3 etc.

product_name:- Name of food items stored in this column.

price:- The cost of food items will be in this column.

In [5]:
# menu data
menu_data = [ ('1', 'palak_paneer', 100),
              ('2', 'chicken_tikka', 150),
              ('3', 'jeera_rice', 120),
              ('4', 'kheer', 110),
              ('5', 'vada_pav', 80),
              ('6', 'paneer_tikka', 180)]
# cols
menu_cols = ['product_id', 'product_name', 'price']
# create the menu dataframe
menu_df = spark.createDataFrame(data = menu_data, schema = menu_cols)
# view the data
menu_df.show()

+----------+-------------+-----+
|product_id| product_name|price|
+----------+-------------+-----+
|         1| palak_paneer|  100|
|         2|chicken_tikka|  150|
|         3|   jeera_rice|  120|
|         4|        kheer|  110|
|         5|     vada_pav|   80|
|         6| paneer_tikka|  180|
+----------+-------------+-----+



##### 3. Members dataframe:- This dataframe has details of those customers who hold any membership from this restaurant. The columns in this dataframe are- customer_id and join_date. The join_date column has the membership purchase date.

In [6]:
# create member data
members_data = [ ('A', '2021-01-07'),
                 ('B', '2021-01-09')]
# cols
members_cols = ["customer_id", "join_date"]
# create the member's dataframe
members_df = spark.createDataFrame(data = members_data, schema = members_cols)
# view the data
members_df.show()

+-----------+----------+
|customer_id| join_date|
+-----------+----------+
|          A|2021-01-07|
|          B|2021-01-09|
+-----------+----------+



#### Solving Problem Statements of Restaurants using PySpark

#### Question 01:- What is the total amount each customer spent at the restaurant?

In [7]:
total_spent_df = (sales_df.join(menu_df, 'product_id') 
                         .groupBy('customer_id').agg({'price':'sum'}) 
                         .withColumnRenamed('sum(price)', 'total_spent_amounts') 
                         .orderBy('customer_id'))

##### Approach:-

First, we will join the sales and menu PySpark dataframe based on the ‘product_id’ column.

Then apply the group by function on ‘customer_id’ and do the sum using the aggregate operation.

Now we will rename the ‘sum(price)’ column with ‘total_spent_amounts’ and use the orderBy function to sort it.

##### Output:-

Customer ‘A’ has spent 760 rupees.

Customer ‘B’ has spent 740 rupees.

Customer ‘C’ has paid 340 rupees.

In [8]:
total_spent_df.show() 

+-----------+-------------------+
|customer_id|total_spent_amounts|
+-----------+-------------------+
|          A|                760|
|          B|                740|
|          C|                340|
+-----------+-------------------+



#### Question 02:- How many days has each customer visited the restaurant?

In [ ]:
# solution:-
res_visit_df = sales_df.groupBy('customer_id').agg(F.countDistinct('order_date')).withColumnRenamed('count(order_date)', 'visit_days')

In [ ]:
res_visit_df.show()

##### Approach:-

First, we will perform groupby on sales data based on ‘customer_id’.

Afterward, perform the ‘countDistinct’ aggregation operation on the ‘order_date’ columns to get each customer’s count of dates.

Finally, we will rename the ‘count(order_date)’ column with ‘visit_days’.


##### Output:-

Customer ‘A’ has visited the restaurant for four days.

Customer ‘B’ has seen the restaurant for six days.

Customer ‘C’ has visited for two days.

#### Question 03:- What was each customer’s first item from the menu?

In [ ]:
# create windowspec
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number,dense_rank,count
windowSpec  = Window.partitionBy("customer_id").orderBy("order_date")
windowSpec

In [ ]:
items_purchased_df = sales_df.withColumn("dense_rank", dense_rank().over(windowSpec)).filter("dense_rank == 1").join(menu_df, 'product_id').select('customer_id', 'product_name').orderBy('customer_id')

In [ ]:
items_purchased_df.show()

#### Approach:-

1. We are going to use the windows function to solve this problem using PySpark. We have created the window spec in which we have partitioned the data based on ‘customer_id’ and sorted the data using the ‘order_date’ column.

2. Now we use the dense_rank() window function to create the ranking as we may have the chance that the customer has purchased more than one item in his first visit.

3. We will apply the filter function to get the first purchase items data. Then we will join the menu data based on ‘product_id’ and select the columns which need to display and sort this data based on ‘customer_id’.

#### Output:-

1. Customer ‘A’ has purchased two items, chicken_tikka, and palak_paneer.

2. Customer ‘B’ has purchased only one food item, chicken_tikka.

3. Customer ‘C’ also purchased two items, palak_paneer, and jeera_rice.

In [ ]:
import pyspark
 
from pyspark.sql import *
 
spark_app = SparkSession.builder.appName('_').getOrCreate()
 
 
students =[(4,'sravan',23,'PHP','Testing'),
           (4,'sravan',23,'PHP','Testing'),
           (46,'mounika',22,'.NET','HTML'),
           (4,'deepika',21,'Oracle','HTML'),
           (46,'mounika',22,'Oracle','Testing'),
           (12,'chandrika',22,'Hadoop','C#'),
           (12,'chandrika',22,'Oracle','Testing'),
           (4,'sravan',23,'Oracle','C#'),
           (4,'deepika',21,'PHP','C#'),
           (46,'mounika',22,'.NET','Testing')
              ]
 
dataframe_obj = spark_app.createDataFrame( students,['subject_id','name','age','technology1','technology2'])
 
print("----------Actual DataFrame----------")
dataframe_obj.show()
 
# import the Window Function
from pyspark.sql.window import Window
 
#import the dense_rank from pyspark.sql.functions
from pyspark.sql.functions import dense_rank
 
#partition the dataframe based on the values in technology1 column and
#order the rows in each partition based on subject_id column
partition = Window.partitionBy("technology1").orderBy('subject_id')
 
print("----------Partitioned DataFrame----------")
 
#Now mention dense_rank for each row in DENSE RANK column
dataframe_obj.withColumn("DENSE RANK",dense_rank().over(partition)).show()

###### Explanation
The first output represents the actual data present in the DataFrame.

In the second output, the partition is done based on the technology1 column.

##### Total partitions: – 4

##### Partition 1:
.NET occurred two times in the first partition. The values in the subject_id column are the same. Hence the dense_rank() function returns the same rank-1 for both .NET values.

##### Partition 2:
Hadoop occurred one time in the second partition. So, the rank is 1.

##### Partition 3:
Oracle occurred four times in the third partition.
For the first two Oracle, the subject_id values are the same, so for both, rank-1 is assigned.
For the third Oracle, the rank is 2.
For the last Oracle, the rank is 3.

##### Partition 4:
PHP occurred three times in the fourth partition, and the values in the subject_id column are the same for all. Hence Rank is 1 for all PHP.

In [ ]:
# import the Window Function
from pyspark.sql.window import Window
 
#import the dense_rank from pyspark.sql.functions
from pyspark.sql.functions import dense_rank
 
#partition the dataframe based on the values in technology2 column and
#order the rows in each partition based on subject_id column
partition = Window.partitionBy("technology2").orderBy('subject_id')
 
print("----------Partitioned DataFrame----------")
 
#Now mention dense_rank for each row in DENSE RANK column
dataframe_obj.withColumn("DENSE RANK",dense_rank().over(partition)).show()

In [ ]:
# importing pyspark
from pyspark.sql.window import Window
import pyspark

# importing sparksession
from pyspark.sql import SparkSession

# creating a sparksession object
# and providing appName
spark = SparkSession.builder.appName("pyspark_window").getOrCreate()

# sample data for dataframe
sampleData = (("Ram", 28, "Sales", 3000),
			("Meena", 33, "Sales", 4600),
			("Robin", 40, "Sales", 4100),
			("Kunal", 25, "Finance", 3000),
			("Ram", 28, "Sales", 3000),
			("Srishti", 46, "Management", 3300),
			("Jeny", 26, "Finance", 3900),
			("Hitesh", 30, "Marketing", 3000),
			("Kailash", 29, "Marketing", 2000),
			("Sharad", 39, "Sales", 4100)
			)

# column names for dataframe
columns = ["Employee_Name", "Age","Department", "Salary"]

# creating the dataframe df
df = spark.createDataFrame(data=sampleData,schema=columns)

# importing Window from pyspark.sql.window

# creating a window
# partition of dataframe
windowPartition = Window.partitionBy("Department").orderBy("Age")

# print schema
df.printSchema()

# show df
df.show()


In [ ]:
# importing cume_dist()
# from pyspark.sql.functions
from pyspark.sql.functions import cume_dist

# applying window function with
# the help of DataFrame.withColumn
df.withColumn("cume_dist",cume_dist().over(windowPartition)).show()


In [ ]:
# importing lag() from pyspark.sql.functions
from pyspark.sql.functions import lag

df.withColumn("Lag", lag("Salary", 2).over(windowPartition)).show()


In [ ]:
# importing row_number() from pyspark.sql.functions
from pyspark.sql.functions import row_number

# applying the row_number() function
df.withColumn("row_number",row_number().over(windowPartition)).show()


In [ ]:
# importing rank() from pyspark.sql.functions
from pyspark.sql.functions import rank

# applying the rank() function
df.withColumn("rank", rank().over(windowPartition)).show()


In [ ]:
# importing percent_rank() from pyspark.sql.functions
from pyspark.sql.functions import percent_rank

# applying the percent_rank() function
df.withColumn("percent_rank",percent_rank().over(windowPartition)).show()


In [ ]:
# importing dense_rank() from pyspark.sql.functions
from pyspark.sql.functions import dense_rank

# applying the dense_rank() function
df.withColumn("dense_rank",dense_rank().over(windowPartition)).show()


##### Aggregate function
An aggregate function or aggregation function is a function where the values of multiple rows are grouped to form a single summary value. The definition of the groups of rows on which they operate is done by using the SQL GROUP BY clause. E.g. AVERAGE, SUM, MIN, MAX, etc. 

##### Creating Dataframe for demonstration:

Before we start with these functions, we will create a new DataFrame that contains employee details like Employee_Name, Department, and Salary. After creating the DataFrame we will apply each Aggregate function on this DataFrame.

In [ ]:
# importing pyspark
import pyspark

# importing sparksessio
from pyspark.sql import SparkSession

# creating a sparksession
# object and providing appName
spark = SparkSession.builder.appName("pyspark_window").getOrCreate()

# sample data for dataframe
sampleData = (("Ram", "Sales", 3000),
			("Meena", "Sales", 4600),
			("Robin", "Sales", 4100),
			("Kunal", "Finance", 3000),
			("Ram", "Sales", 3000),
			("Srishti", "Management", 3300),
			("Jeny", "Finance", 3900),
			("Hitesh", "Marketing", 3000),
			("Kailash", "Marketing", 2000),
			("Sharad", "Sales", 4100)
			)

# column names for dataframe
columns = ["Employee_Name", "Department", "Salary"]

# creating the dataframe df
df3 = spark.createDataFrame(data=sampleData,schema=columns)

# print schema
df3.printSchema()

# show df
df3.show()


In [ ]:
# importing window from pyspark.sql.window
from pyspark.sql.window import Window

# importing aggregate functions
# from pyspark.sql.functions
from pyspark.sql.functions import col,avg,sum,min,max,row_number

# creating a window partition of dataframe
windowPartitionAgg = Window.partitionBy("Department")

# applying window aggregate function
# to df3 with the help of withColumn

# this is average(),#this is min(),#this is max()
df3.withColumn("Avg", avg(col("salary")).over(windowPartitionAgg)).withColumn("Sum", sum(col("salary")).over(windowPartitionAgg)).withColumn("Min", min(col("salary")).over(windowPartitionAgg)).withColumn("Max",max(col("salary")).over(windowPartitionAgg)).show()


##### Question 04:- Find out the most purchased item from the menu and how many times the customers purchased it.

In [ ]:
# code
from pyspark.sql.functions import *
most_purchased_df = sales_df.join(menu_df, 'product_id').groupBy('product_id', 'product_name').agg(count('product_id').alias('product_count')).orderBy('product_count', ascending=0) .drop('product_id').limit(2)

In [ ]:
most_purchased_df.show()

##### Approach:-

We will first join the sales and menu data to solve this problem.

Then perform grouping based on ‘product_id’ and ‘product_name’.

Next, we will perform a ‘count’ aggregate operation on ‘product_id’ to count the items and then sort the data using ‘product_id’ and limit the result to show only the first row.

##### Output:-

The most purchased item is ‘jeera_rice,’ purchased seven times by customers.

#####  Question 05:- Which item was the most popular for each customer?

In [ ]:
# code:-

popular_item_df = sales_df.join(menu_df, 'product_id').groupBy('customer_id', 'product_name').agg(count('product_id').alias('count_orders')).withColumn('dense_rank', dense_rank().over(Window.partitionBy("customer_id") .orderBy(col("count_orders").desc()))).filter('dense_rank=1').drop('dense_rank').show()

##### Approach:-

We need to join the sales and menu data to find the most popular items for each customer. Then apply the groupby function to create different groups using ‘customer_id’ and ‘product_name’.

Now we use the count aggregate operation to count the order given by each customer for different items.

Afterward, we use the dense rank window function to rank the items based on orders_count. Then use the filter function to get the data where ‘dense_rank’ equals one. Finally, drop the ‘dense_rank’ column.

##### Output:-

Customer ‘A’ has ordered ‘jeera_rice’ three times.

Customer ‘B’ has ordered ‘chicken_tikka’, ‘jeera_rice’, and palak_paneer’, all three items twice.

Customer ‘C’ has ordered ‘jeera_rice’ twice.

##### Question 06:- Which item was ordered first by the customer after becoming a restaurant member?

In [ ]:
windowSpec = Window.partitionBy("customer_id").orderBy("order_date")

items_after_member_df = sales_df.join(members_df, 'customer_id').filter(sales_df.order_date >= members_df.join_date).withColumn('dense_rank', dense_rank().over(windowSpec)).filter('dense_rank = 1') .join(menu_df,'product_id') .select('customer_id', 'order_date', 'product_name')

In [ ]:
items_after_member_df.show()

#### Approach:-

1. We use all three sales, menu, and members data frames to solve this problem. First, we will join the sales and members dataframe.

2. Then filter out the records where the ‘order_date’ value is more than the member ‘join_date’ value. Now we have the data points after the users have become a member.

3. We will apply the dense_rank function to rank the data and take out those data points where rank equals one.

4. Finally, we join the menu dataframe with the filtered data and select the required columns.

#### Output:-

1. Customer ‘A’ has ordered the ‘chicken_tikka’ as the first item after becoming a member.

2. Customer ‘B’ has purchased ‘palak_paneer’ as the first item.

##### Question 07:- Which item was purchased before the customer became a member?

In [ ]:
windowSpec = Window.partitionBy("customer_id").orderBy(col("order_date").desc())
items_before_member_df = sales_df.join(members_df, 'customer_id').filter(sales_df.order_date < members_df.join_date).withColumn('dense_rank', dense_rank().over(windowSpec)).filter('dense_rank = 1').join(menu_df,'product_id').select('customer_id', 'order_date', 'product_name')

In [ ]:
items_before_member_df.show()

#### Approach:-

All the steps of the above problem will be the same for this problem with little changes. In the filter condition, we select those data where order_date is less than the member’s join date.

Then in the window function, sort the data in descending order and select the required columns.

##### Output:-

Customer ‘A’ has ordered ‘palak paneer’ and ‘chicken tikka’ before becoming a member.

Customer ‘B’ has ordered ‘palak paneer’.

##### Question 08:- What is the total items and amount spent for each member before they became a member?

In [ ]:
from pyspark.sql.functions import countDistinct

total_items_spent_df = sales_df.join(menu_df, 'product_id').join(members_df, 'customer_id').filter(sales_df.order_date < members_df.join_date).groupBy('customer_id').agg(countDistinct('product_id').alias('item_counts'),sum('price').alias('total_amount'))

In [ ]:
total_items_spent_df.show()

#### Approach:-

1. We use all three sales, menu, and members data frames to solve this problem using PySpark functions. First, we will join all three data frames.

2. Then apply the filter function to select those rows where ‘order_date is less than ‘join_date’. Doing this allows us to get data when the customer did not become a member.

3. After that, we use the groupby function based on ‘customer_id’ and then use aggregate operations on it to select different items and find the total spent amount.

##### output:-

Customer ‘A’ has purchased two items, and he has spent 250 rupees on them.

Customer ‘B’ also purchased two food items and spent 400 rupees on them.

##### Question 09:- If each rupee spent equates to 10 points and item ‘jeera_rice’ has a 2x points multiplier, find out how many points each customer would have.

In [ ]:
from pyspark.sql.functions import when
earned_points_df = sales_df.join(menu_df, 'product_id').withColumn('points', when(col('product_id') == 3, col('price')*20).otherwise(col('price')*10)).groupBy('customer_id').agg(sum('points').alias('rewards_points'))

In [ ]:
earned_points_df.show()

#### Approach:-

1. First, we joined the sales and menu dataframe and created the new column using the ‘withColumn’ function using PySpark functions.

2. In this, we use condition when the ‘product_id’ is equal to three, i.e. the product id of item’ jeera_rice’, then we multiply the price col values with 20, and the other will get multiplied by 10.

3. Last, we perform grouping on the data and then use aggregate operation on ‘points’ data to calculate total rewards points for each customer.

##### Output:-

Customer ‘A’ has the highest reward points, i.e. 11,200.

Customer ‘B’ has a total of 9800 reward points.

The customer ‘C’ has scored a total of 5800 reward points.

##### Question 10:- Create the complete table with all data and columns like customer_id, order_date, product_name, price, and member(Y/N).

In [ ]:
all_data_df = sales_df.join(menu_df, 'product_id', 'left').join(members_df, 'customer_id', 'left').withColumn('member', when(col('order_date') < col('join_date'), 'N').when(col('order_date')>= col('join_date'), 'Y').otherwise('N')).drop('product_id', 'product_name', 'join_date')

In [ ]:
all_data_df.show()

##### Approach:-

1. We joined all three data frames using ‘left’ join and then used the condition, when the ‘order_date’ is less than ‘join_date’, then populate ‘N’ in the ‘member’ column. If ‘order_date’ is greater or equal to ‘joined_date’, then set the value with ‘Y’, and otherwise ‘N’.

2. Finally dropped the other columns which are not required.

##### Output:-

After applying the transformations, we get all the required details as a final output.

##### Question 11:- We also require further information about the ranking of customer products. The owner does not need the ranking for non-member purchases, so he expects null ranking values for the records when customers still need to be part of the membership program.

In [ ]:
ranking_final_df = sales_df.join(menu_df, 'product_id', 'left').join(members_df, 'customer_id', 'left').withColumn('is_member', when(col('order_date') < col('join_date'), 'N').when(col('order_date')>= col('join_date'), 'Y').otherwise('N')).withColumn('rank', when(col('is_member') == 'N', None).when(col('is_member') == 'Y', rank().over(Window.partitionBy('customer_id','is_member').orderBy('order_date'))).otherwise(0))

In [ ]:
ranking_final_df.show()

#### Approach:-

1. All the steps of the above problem will be the same for this problem, with slight changes. For populating null values against non-members, we will use the rank function.

2. We check if the ‘is_member’ value is ‘None’, then the rank column will have ‘null’ values corresponding to that row; otherwise, populate the rank for that particular data point.

##### Output:-

After applying the transformations, we get all the required details as a final output.

##### Conclusion
In this article, we worked on a case study using restaurant data. We find some hidden insights from the data that case increases the sales of items for the restaurants using PySpark. Now the owner can understand which items are popular and which need more focus. Based on this insight, the owner can increase his sales in many ways by giving good discounts to the user without worrying about losses.

##### Some Important Takeaways from this article are as follows:-

1. We worked with real-world data and gathered some insightful information that can benefit businesses in various ways.

2. We worked with different types of PySpark functions, including join, groupBy, withColumn, withColumnRenamed, partitionBy, and Windows functions like rank and dense rank, to provide answers.

3. Furthermore, we talked about various step-by-step solutions to the problem using PySpark functions. We also analyze the output at the end of each problem statement.

##### Example 1: Retrieving all the Data from the Dataframe using collect().

After creating the Dataframe, for retrieving all the data from the dataframe we have used the collect() action by writing df.collect(), this will return the Array of row type, in the below output shows the schema of the dataframe and the actual created Dataframe.

In [ ]:
# importing necessary libraries
from pyspark.sql import SparkSession

# function to create new SparkSession
def create_session():
    spk = SparkSession.builder.appName("Corona_cases_statewise.com").getOrCreate()
    return spk

# function to create RDD
def create_RDD(sc_obj,data):
    df = sc.parallelize(data)
    return df


if __name__ == "__main__":
	
    input_data = [("Uttar Pradesh",122000,89600,12238),
		("Maharashtra",454000,380000,67985),
		("Tamil Nadu",115000,102000,13933),
		("Karnataka",147000,111000,15306),
		("Kerala",153000,124000,5259)]

# calling function to create SparkSession
spark = create_session()

# creating spark context object
sc = spark.sparkContext

# calling function to create RDD
rd_df = create_RDD(sc,input_data)

schema_lst = ["State","Cases","Recovered","Deaths"]

# creating the dataframe using createDataFrame function
df = spark.createDataFrame(rd_df,schema_lst)
# printing schema of the dataframe and showing the dataframe
df.printSchema()
df.show()
	
# retrieving the data from the dataframe using collect()
df2= df.collect()
print("Retrieved Data is:-")
print(df2)

##### Example 2: Retrieving Data of specific rows using collect().

After creating the Dataframe, we have retrieved the data of 0th row Dataframe using collect() action by writing print(df.collect()[0][0:]) respectively in this we are passing row and column after collect(), in the first print statement we have passed row and column as [0][0:] here first [0] represents the row that we have passed 0 and second [0:] this represents the column and colon(:) is used to retrieve all the columns, in short, we have retrieve the 0th row with all the column elements.

In [ ]:
# importing necessary libraries
from pyspark.sql import SparkSession

# function to create new SparkSession
def create_session():
    spk = SparkSession.builder \
	.appName("Corona_cases_statewise.com") \
	.getOrCreate()
    return spk

# function to create RDD
def create_RDD(sc_obj,data):
    df = sc.parallelize(data)
    return df


if __name__ == "__main__":
	
    input_data = [("Uttar Pradesh",122000,89600,12238),
		("Maharashtra",454000,380000,67985),
		("Tamil Nadu",115000,102000,13933),
		("Karnataka",147000,111000,15306),
		("Kerala",153000,124000,5259)]

# calling function to create SparkSession
spark = create_session()

# creating spark context object
sc = spark.sparkContext

# calling function to create RDD
rd_df = create_RDD(sc,input_data)

schema_lst = ["State","Cases","Recovered","Deaths"]

# creating the dataframe using createDataFrame function
df = spark.createDataFrame(rd_df,schema_lst)

# printing schema of the dataframe and showing the dataframe
df.printSchema()
df.show()
	
print("Retrieved Data is:-")
	
# Retrieving data from 0th row
print(df.collect()[0][0:])


##### Example 3: Retrieve data of multiple rows using collect().

After creating the Dataframe, we are retrieving the data of the first three rows of the dataframe using collect() action with for loop, by writing for row in df.collect()[0:3], after writing the collect() action we are passing the number rows we want [0:3], first [0] represents the starting row and using “:” semicolon and [3] represents the ending row till which we want the data of multiple rows.

Here is the number of rows from which we are retrieving the data is 0,1 and 2 the last index is always excluded i.e, 3.

In [ ]:
# importing necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# function to create new SparkSession
def create_session():
    spk = SparkSession.builder \
	.appName("Corona_cases_statewise.com") \
	.getOrCreate()
    return spk

# function to create RDD
def create_RDD(sc_obj,data):
    df = sc.parallelize(data)
    return df


if __name__ == "__main__":
	
    input_data = [("Uttar Pradesh",122000,89600,12238),
		("Maharashtra",454000,380000,67985),
		("Tamil Nadu",115000,102000,13933),
		("Karnataka",147000,111000,15306),
		("Kerala",153000,124000,5259)]

# calling function to create SparkSession
spark = create_session()

# creating spark context object
sc = spark.sparkContext

# calling function to create RDD
rd_df = create_RDD(sc,input_data)

schema_lst = ["State","Cases","Recovered","Deaths"]

# creating the dataframe using createDataFrame function
df = spark.createDataFrame(rd_df,schema_lst)

# showing the dataframe and schema
df.printSchema()
df.show()
	
print("Retrieved Data is:-")
	
# Retrieving multiple rows using collect() and for loop
for row in df.collect()[0:3]:
	print((row["State"]),",",str(row["Cases"]),",",
		str(row["Recovered"]),",",str(row["Deaths"]))


##### Example 4: Retrieve data from a specific column using collect().

After creating the Dataframe, we are retrieving the data of ‘Cases’ column using collect() action with for loop. By iterating the loop to df.collect(), that gives us the Array of rows from that rows we are retrieving and printing the data of ‘Cases’ column by writing print(col[“Cases”]);

As we are getting the rows one by iterating for loop from Array of rows, from that row we are retrieving the data of “Cases” column only. By  writing print(col[“Cases”]) here from each row we are retrieving the data of ‘Cases’ column by passing ‘Cases’ in col.

In [ ]:
# importing necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# function to create new SparkSession
def create_session():
    spk = SparkSession.builder \
	.appName("Corona_cases_statewise.com") \
	.getOrCreate()
    return spk

# function to create RDD
def create_RDD(sc_obj,data):
    df = sc.parallelize(data)
    return df

if __name__ == "__main__":
	
    input_data = [("Uttar Pradesh",122000,89600,12238),
		("Maharashtra",454000,380000,67985),
		("Tamil Nadu",115000,102000,13933),
		("Karnataka",147000,111000,15306),
		("Kerala",153000,124000,5259)]

# calling function to create SparkSession
spark = create_session()

# creating spark context object
sc = spark.sparkContext

# calling function to create RDD
rd_df = create_RDD(sc,input_data)

schema_lst = ["State","Cases","Recovered","Deaths"]

# creating the dataframe using createDataFrame function
df = spark.createDataFrame(rd_df,schema_lst)

# showing the dataframe and schema
df.printSchema()
df.show()
	
print("Retrieved Data is:-")
	
# Retrieving data from the "Cases" column
for col in df.collect():
	print(col["Cases"])


##### Example 5: Retrieving the data from multiple columns using collect().

After creating the dataframe, we are retrieving the data of multiple columns which include “State”, “Recovered” and “Deaths”.

For retrieving the data of multiple columns, firstly we have to get the Array of rows which we get using df.collect() action now iterate the for loop of every row of Array, as by iterating we are getting rows one by one so from that row we are retrieving the data of “State”, “Recovered” and “Deaths” column from every column and printing the data by writing, print(col[“State”],”,”,col[“Recovered”],”,”,col[“Deaths”])

In [ ]:
# importing necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# function to create new SparkSession
def create_session():
    spk = SparkSession.builder \
	.appName("Corona_cases_statewise.com") \
	.getOrCreate()
    return spk

# function to create RDD
def create_RDD(sc_obj,data):
    df = sc.parallelize(data)
    return df


if __name__ == "__main__":
	
 input_data = [("Uttar Pradesh",122000,89600,12238),
		("Maharashtra",454000,380000,67985),
		("Tamil Nadu",115000,102000,13933),
		("Karnataka",147000,111000,15306),
		("Kerala",153000,124000,5259)]

# calling function to create SparkSession
spark = create_session()

# creating spark context object
sc = spark.sparkContext

# calling function to create RDD
rd_df = create_RDD(sc,input_data)

schema_lst = ["State","Cases","Recovered","Deaths"]

# creating the dataframe using createDataFrame function
df = spark.createDataFrame(rd_df,schema_lst)

# showing the dataframe and schema
df.printSchema()
df.show()
	
print("Retrieved Data is:-")
	
# Retrieving data of the "State",
# "Recovered" and "Deaths" column
for col in df.collect():
	print(col["State"],",",col["Recovered"],",",col["Deaths"])
